In [1]:
import pandas as pd
import os
import random
from tqdm import tqdm
import pickle
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
import matplotlib.pyplot as plt
import seaborn as sns

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
regenerate_context_data = True
sequence_augmentation = True
whole_session_context = False
model_test_run = False
data_autoencoder = False
pad_to_window_size = True
feat_engg = True

PATH_TO_LOAD = '../data/04_Merged'
if feat_engg:
    combined_context_path = '../data/06_context_feat_engg/data_featue_engineering.csv'
    augmentation_folder = 'featengg/' if sequence_augmentation else 'non_aug/'
else:
    combined_context_path = '../data/05_Interaction_Sequences/context.csv'
    augmentation_folder = 'fix/' if sequence_augmentation else 'non_aug/'

window = 100 #seconds

base_path = '../datasets/sequential/'
if model_test_run:
    augmentation_folder = 'test/aug/' if sequence_augmentation else 'test/non_aug/'

sequence_context_path = f'{base_path}{augmentation_folder}parameters/sequence_context.csv'
parameter_path = f'{base_path}{augmentation_folder}parameters'
train_session_path = f'{base_path}{augmentation_folder}parameters/train_sessions.pkl'
test_session_path = f'{base_path}{augmentation_folder}parameters/test_sessions.pkl'
train_dynamic_context_path = f'{base_path}{augmentation_folder}dynamic_context/train.csv'
test_dynamic_context_path = f'{base_path}{augmentation_folder}dynamic_context/test.csv'
train_static_context_path = f'{base_path}{augmentation_folder}static_context/train.csv'
test_static_context_path = f'{base_path}{augmentation_folder}static_context/test.csv'
train_sequence_path = f'{base_path}{augmentation_folder}seq/train.tsv'
test_sequence_path = f'{base_path}{augmentation_folder}seq/test.tsv'
train_dense_static_context_path = f'{base_path}{augmentation_folder}dense_static_context/train.csv'
test_dense_static_context_path = f'{base_path}{augmentation_folder}dense_static_context/test.csv'

In [3]:
all_columns = ['index', 'avg_irradiation', 'steering_speed', 'temperature_out', 'hour',
       'month', 'odometer', 'light_sensor_rear', 'light_sensor_front',
       'temperature_in', 'KBI_speed', 'soc', 'ESP_speed', 'latitude',
       'longitude', 'seatbelt_codriver', 'seatbelt_rear_l', 'seatbelt_rear_m',
       'seatbelt_rear_r', 'CHA_ESP_drive_mode', 'CHA_MO_drive_mode',
       'rain_sensor', 'street_category', 'kickdown', 'altitude',
       'driving_program', 'datetime', 'session', 'Label', 'ID',
       'FunctionValue', 'domain', 'BeginTime', 'time_second',
       'distance_driven', 'ts_normalized', 'weekday']

selected = [ 'avg_irradiation', 'steering_speed', 'temperature_out', 'hour',
       'month', 'light_sensor_rear', 'light_sensor_front',
       'temperature_in', 'KBI_speed', 'soc', 'latitude',
       'longitude', 'seatbelt_codriver', 'seatbelt_rear_l',
       'seatbelt_rear_r', 'street_category', 'altitude',
       'datetime', 'session', 'time_second',
       'distance_driven', 'weekday'
]

bad_quality = ['CHA_ESP_drive_mode', 
             'CHA_MO_drive_mode',
             'rain_sensor',
             'kickdown',
             'ESP_speed',
             'seatbelt_rear_m',
            'driving_program',
            'ts_normalized'
             ]

dynamic_context_var = ['avg_irradiation', 'steering_speed', 'temperature_out', 
                       'light_sensor_rear', 'light_sensor_front', 
                       'temperature_in', 'KBI_speed', 
                       'latitude','longitude', 'altitude'] # todo remove these features in the future
if feat_engg:
       cat_static_context_var = ['car_id', 'month', 'weekday', 'hour', 'season', 'seatbelt_codriver', 'seatbelt_rear_l', # categorical static context
                            'seatbelt_rear_r',  'street_category']
else:
       cat_static_context_var = ['car_id', 'month', 'weekday', 'hour', 'seatbelt_codriver', 'seatbelt_rear_l', # categorical static context
                            'seatbelt_rear_r',  'street_category']
dense_static_context_var =  ['distance_driven_benchmark', 'soc', 'time_second']  # dense static context
status_static_context_var = ['ess_status', 'current_drive_mode', 'current_clima_mode', 'current_media_source', # status static context
                     'nav_guidance_status', 'proximity_to_parking_spot', 'phone_status',
                     'bluetooth_connected', 'phone_os',
                     'new_bluetooth_device_to_pair']
#todo i feel street category is higly fluctuating. might be better to ignore
vehicles = ['SEB880','SEB882','SEB883','SEB885','SEB888','SEB889']

In [4]:
context_data = pd.read_csv(combined_context_path, parse_dates=['datetime'], index_col=0, low_memory=False)
context_data_filtered = context_data[context_data['distance_driven'] != 0]
context_data_filtered['distance_driven_benchmark'] = context_data_filtered.groupby('session')['distance_driven'].transform(lambda x: x - x.min())
context_data['distance_driven_benchmark'] = context_data_filtered['distance_driven_benchmark']
context_data['distance_driven_benchmark'].fillna(0, inplace=True)
context_data

/var/folders/n8/1ntcg5gj7nvf5wh_s2w95yh80000gn/T/ipykernel_3738/4045549043.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  context_data_filtered['distance_driven_benchmark'] = context_data_filtered.groupby('session')['distance_driven'].transform(lambda x: x - x.min())


,session,datetime,Label,avg_irradiation,steering_speed,temperature_out,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,...,current_clima_mode,current_media_source,nav_guidance_status,proximity_to_parking_spot,phone_status,bluetooth_connected,season,phone_os,new_bluetooth_device_to_pair,distance_driven_benchmark
0,7.0,2022-09-07 20:08:33,NaN,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00,...,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn,unknown,no bluetooth device to pair,0.0
1,7.0,2022-09-07 20:08:34,NaN,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00,...,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn,unknown,no bluetooth device to pair,0.0
2,7.0,2022-09-07 20:08:35,NaN,0.000000,0.000000,25.0,0.0,0.000000,0.0,0.00,...,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn,unknown,no bluetooth device to pair,0.0
3,7.0,2022-09-07 20:08:36,NaN,0.000000,0.000000,25.0,0.0,0.000000,27.0,0.00,...,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn,unknown,no bluetooth device to pair,0.0
4,7.0,2022-09-07 20:08:36,NaN,0.000000,0.000000,25.0,0.0,0.000000,27.0,0.00,...,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn,unknown,no bluetooth device to pair,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10124879,5399.0,2023-03-31 17:09:13,NaN,53.755324,0.000000,13.5,17600.0,6126.000000,28.5,92.48,...,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Spring,unconnected,no bluetooth device to pair,145.0
10124880,5399.0,2023-03-31 17:09:14,NaN,50.000000,0.000000,13.5,17200.0,5920.781157,28.5,93.12,...,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Spring,unconnected,no bluetooth device to pair,145.0
10124881,5399.0,2023-03-31 17:09:15,NaN,56.226313,15.000000,13.5,18000.0,5699.908827,28.5,93.44,...,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Spring,unconnected,no bluetooth device to pair,145.0
10124882,5399.0,2023-03-31 17:09:16,NaN,60.000000,0.000000,13.5,17600.0,5437.218229,28.5,93.76,...,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Spring,unconnected,no bluetooth device to pair,145.0


In [5]:
context_data_list = []
for session in tqdm(context_data.session.unique().tolist()):
    context_data_curr = context_data[context_data['session']== session]
    context_data_curr['distance_driven_benchmark'] = context_data_curr['distance_driven_benchmark'].replace(0, method='ffill')
    context_data_list.append(context_data_curr)
context_data = pd.concat(context_data_list, axis=0)

  0%|          | 0/1704 [00:00<?, ?it/s]/var/folders/n8/1ntcg5gj7nvf5wh_s2w95yh80000gn/T/ipykernel_3738/540319804.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  context_data_curr['distance_driven_benchmark'] = context_data_curr['distance_driven_benchmark'].replace(0, method='ffill')
100%|██████████| 1704/1704 [00:16<00:00, 101.83it/s]


In [6]:

# import matplotlib.pyplot as plt

# # Assuming df is your DataFrame containing the 'session' and 'distance_driven_benchmark' columns

# # Group the DataFrame by 'session'
# grouped_df = context_data.groupby('session')

# # Iterate over each group and create line plots
# for session, group in grouped_df:
#     plt.figure(figsize=(4, 3)) 
#     plt.figure()  # Create a new figure for each session
#     plt.plot(group.index, group['time_second'], marker='o', linestyle='-')
#     plt.title(f'Distance Driven Benchmark for Session {session}')
#     plt.xlabel('Index')
#     plt.ylabel('Distance Driven Benchmark')
#     plt.grid(True)
#     plt.show()


In [7]:
with open(train_session_path, 'rb') as pickle_file:
    train_sessions = pickle.load(pickle_file)
with open(test_session_path, 'rb') as pickle_file:
    test_sessions = pickle.load(pickle_file)
context_data = context_data[context_data['session'].isin(train_sessions + test_sessions)]

train_sequence = pd.read_csv(train_sequence_path, sep='\t', low_memory=False)
test_sequence = pd.read_csv(test_sequence_path, sep='\t', low_memory=False)
selected_sequence = pd.concat([train_sequence, test_sequence], axis=0).sort_values(['session', 'window_id'])

# selected_sequence = pd.read_csv(sequence_context_path, parse_dates=['datetime'], index_col=0)
# selected_sequence['session'] = selected_sequence['session'].astype(int)
# min_datetime_indices = selected_sequence.groupby('session')['datetime'].idxmin()
# selected_sequence = selected_sequence.drop(min_datetime_indices)
# selected_sequence.reset_index(drop=True, inplace=True)

# selected_dfs = []
# for session in tqdm(selected_sequence['session'].unique().tolist()):
#     selected_sequence_curr = selected_sequence[selected_sequence['session']==session]
#     context_data_curr = context_data[context_data['session']==session]
#     context_data_curr = context_data_curr[context_data_curr['datetime']<=selected_sequence_curr['timestamp_target_interaction'].max()]
#     selected_dfs.append(context_data_curr)
# training_sequence_context = pd.concat(selected_dfs, axis=0)
training_sequence_context = context_data

In [8]:
print(len(context_data.session.unique().tolist()), 
      len(training_sequence_context.session.unique().tolist()))

1634 1634


In [9]:
if sequence_augmentation == True:
    augmented_frames = []
    for index, row in tqdm(selected_sequence.iterrows(), total=len(selected_sequence)):
        session = row['session']
        window_id = row['window_id']
        timestamp_target_interaction = row['timestamp_target_interaction']
        training_sequence_context_curr = training_sequence_context[(training_sequence_context['session'] == session) &
                                                    (training_sequence_context['datetime'] <= timestamp_target_interaction)].copy()
        if training_sequence_context_curr.empty:
            print(session, window_id)
        if not whole_session_context and window < len(training_sequence_context_curr):
                training_sequence_context_curr = training_sequence_context_curr.tail(window)
        training_sequence_context_curr['window_id'] = window_id
        augmented_frames.append(training_sequence_context_curr)
        # print(session, window_id, timestamp_target_interaction)
        # break
    training_sequence_context_augmented = pd.concat(augmented_frames, axis=0)
    context_data = training_sequence_context_augmented.reset_index(drop=True)
    context_data['wind_id'] = context_data.groupby(['session', 'window_id']).ngroup()
else:
    # if sequence_augmentation is set to false
    if not whole_session_context:
        context_data = training_sequence_context.groupby('session').tail(window)
    context_data = training_sequence_context.reset_index(drop=True)
    context_data['window_id'] = context_data.groupby('session').ngroup()

100%|██████████| 5883/5883 [04:13<00:00, 23.24it/s]


In [10]:
print('total number of sequence data sessions: ', len(selected_sequence.session.unique().tolist()))
print('total number of Sequence data windows: ', len(train_sequence.window_id.unique().tolist()) + len(test_sequence.window_id.unique().tolist()))
print('total number of context data sessions: ', len(context_data.session.unique().tolist()))
print('total number of context data windows: ', len(context_data.wind_id.unique().tolist()))
#dont be the bothered about the total number of windows.

total number of sequence data sessions:  1634
total number of Sequence data windows:  5883
total number of context data sessions:  1634
total number of context data windows:  5883


In [11]:
# testing_sessions = [16, 25]
# selected_sequence = selected_sequence[selected_sequence['session'].isin(testing_sessions)]
# training_sequence_context = training_sequence_context[training_sequence_context['session'].isin(testing_sessions)]

# window_id = 0
# if sequence_augmentation == True:
#     grouped_selected_sequence = selected_sequence.groupby('session')
#     augmented_frames = []
#     for session, selected_sequence_curr in tqdm(grouped_selected_sequence):
#         for i, row in selected_sequence_curr.iloc[::-1].iterrows():
#             context_filt_curr = training_sequence_context[
#                 (training_sequence_context['session'] == session) &
#                 (training_sequence_context['datetime'] < row['datetime'])].copy()
#             if not whole_session_context:
#                 context_filt_curr = context_filt_curr.tail(window)
#             context_filt_curr['window_id'] = window_id
#             # context_filt_curr['session'] = session
#             augmented_frames.append(context_filt_curr)
#             window_id += 1
#     training_sequence_context_augmented = pd.concat(augmented_frames, axis=0)
#     context_data = training_sequence_context_augmented.reset_index(drop=True)
# else:
#     # if sequence_augmentation is set to false
#     if not whole_session_context:
#         context_data = training_sequence_context.groupby('session').tail(window)
#     context_data = training_sequence_context.reset_index(drop=True)
#     context_data['window_id'] = context_data.groupby('session').ngroup()

In [12]:
dynamic_context = context_data[dynamic_context_var + ['window_id', 'session', 'datetime', 'wind_id']]
print('number of dynamic context session', len(dynamic_context[['window_id', 'session']].drop_duplicates()))

# function to pad first value to fit the window size
if pad_to_window_size:
    df = dynamic_context.copy()
    session_counts = df.groupby('wind_id').size()
    less_than_100 = session_counts[session_counts < window].index.tolist()
    print(f'Number of window with window length less than {window}: ', len(less_than_100))
    window100_dfs = df[~df['wind_id'].isin(less_than_100)]
    empty_df = []
    for window_id in tqdm(less_than_100):
        sub_df = df[df['wind_id'] == window_id]
        rows_to_pad = window - len(sub_df)
        min_datetime_row = sub_df.loc[sub_df['datetime'].idxmin()]
        pad_df = pd.DataFrame(min_datetime_row, df.columns).transpose()
        pad_df = pd.concat([pad_df] * int(rows_to_pad), ignore_index=True, axis=0)

        padded_df = pd.concat([pad_df, sub_df], axis=0).reset_index(drop=True)
        # padded_df['wind_id'] = window_id
        # padded_df['window_id'] = window_id
        # padded_df['wind_id'] = window_id
        empty_df.append(padded_df)
    if empty_df:
        df = pd.concat(empty_df, axis=0).reset_index(drop=True)
        df = pd.concat([df, window100_dfs], axis=0).sort_values(by=['window_id']).reset_index(drop=True)
        session_counts = df.groupby('window_id').size()
        less_than_100 = session_counts[session_counts < window].index.tolist()
        print(f'Number of window with window length less than {window}: ', len(less_than_100))
        dynamic_context = df

number of dynamic context session 5883
Number of window with window length less than 100:  208


100%|██████████| 208/208 [00:01<00:00, 153.39it/s]


Number of window with window length less than 100:  0


In [13]:
dense_static_context = context_data[dense_static_context_var + ['window_id', 'session', 'datetime', 'wind_id']]
dense_static_context = dense_static_context.sort_values(by=['wind_id','datetime'], ascending=False)
dense_static_context = dense_static_context.groupby('wind_id').first()
dense_static_context.reset_index(inplace=True)
dense_static_context = dense_static_context.sort_values(by='wind_id')
dense_static_context

,wind_id,distance_driven_benchmark,soc,time_second,window_id,session,datetime
0,0,2.0,48.55,3138.0,0,7.0,2022-09-07 21:00:51
1,1,49.0,56.65,3297.0,0,16.0,2022-09-09 18:43:10
2,2,44.0,58.10,2150.0,1,16.0,2022-09-09 18:24:03
3,3,18.0,63.50,846.0,2,16.0,2022-09-09 18:02:19
4,4,0.0,56.65,352.0,1,20.0,2022-09-10 17:15:06
...,...,...,...,...,...,...,...
5878,5878,23.0,56.30,891.0,1258,5387.0,2023-03-28 12:54:10
5879,5879,52.0,8.50,1846.0,4620,5388.0,2023-03-28 14:16:00
5880,5880,125.0,60.85,6216.0,4621,5389.0,2023-03-28 18:38:29
5881,5881,41.0,55.45,2391.0,4622,5399.0,2023-03-31 16:19:44


In [14]:
cat_static_context = context_data[cat_static_context_var + ['window_id', 'session', 'datetime', 'wind_id']]
cat_static_context = cat_static_context.groupby('wind_id').apply(lambda x: x.mode().iloc[0]).reset_index(drop=True)
if feat_engg:
    cat_static_context['season'], _ = pd.factorize(cat_static_context['season'])  

    status_static_context = context_data[status_static_context_var + ['window_id', 'session', 'datetime', 'wind_id']]
    for col in status_static_context_var:
        print(col)
        print(sorted(status_static_context[col].unique().tolist()))  
    for col in status_static_context_var:
        status_static_context[col], _ = pd.factorize(status_static_context[col])  
    status_static_context = status_static_context.sort_values(by=['session', 'datetime'])
    latest_indices = status_static_context.groupby('wind_id')['datetime'].idxmax()
    status_static_context_filt = status_static_context.loc[latest_indices]
    status_static_context = status_static_context_filt.reset_index(drop=True)

    static_context = pd.merge(cat_static_context, status_static_context, on=['wind_id', 'window_id', 'session'], how='inner')
else:
    static_context = cat_static_context
static_context = static_context.sort_values(by='wind_id')
print('number of windows', len(dynamic_context.wind_id.unique().tolist()), len(static_context.wind_id.unique().tolist()))
print('number of session', len(dynamic_context.session.unique().tolist()), len(static_context.session.unique().tolist()))
static_context

ess_status
[0, 1]
current_drive_mode
['car/driveMode/0.0', 'car/driveMode/1.0', 'car/driveMode/2.0', 'car/driveMode/3.0']
current_clima_mode
['clima/AC/ECO', 'clima/AC/off', 'clima/AC/on', 'unknown']
current_media_source
['media/selectedSource/Bluetooth', 'media/selectedSource/CarPlay', 'media/selectedSource/Favorite', 'media/selectedSource/Radio', 'media/selectedSource/unavailable']
nav_guidance_status
['navi_active', 'navi_inactive']
proximity_to_parking_spot
['no_parking_spot_closeby', 'parking_spot_closeby']
phone_status
['phone_connected', 'unconnected']
bluetooth_connected
['bluetooth_connected', 'bluetooth_unconnected']
phone_os
['Android', 'ios', 'unconnected', 'unknown']
new_bluetooth_device_to_pair
['new bluetooth device available', 'no bluetooth device to pair']


/var/folders/n8/1ntcg5gj7nvf5wh_s2w95yh80000gn/T/ipykernel_3738/3662061914.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  status_static_context[col], _ = pd.factorize(status_static_context[col])
/var/folders/n8/1ntcg5gj7nvf5wh_s2w95yh80000gn/T/ipykernel_3738/3662061914.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  status_static_context[col], _ = pd.factorize(status_static_context[col])
/var/folders/n8/1ntcg5gj7nvf5wh_s2w95yh80000gn/T/ipykernel_3738/3662061914.py:11: SettingWithCopyWarning: 
A v

number of windows 5883 5883
number of session 1634 1634


,car_id,month,weekday,hour,season,seatbelt_codriver,seatbelt_rear_l,seatbelt_rear_r,street_category,window_id,...,current_drive_mode,current_clima_mode,current_media_source,nav_guidance_status,proximity_to_parking_spot,phone_status,bluetooth_connected,phone_os,new_bluetooth_device_to_pair,datetime_y
0,1.0,9.0,2.0,21.0,0,1.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,2022-09-07 21:00:51
1,1.0,9.0,4.0,18.0,0,0.0,1.0,0.0,7.0,0.0,...,0,0,1,0,0,0,0,0,0,2022-09-09 18:43:10
2,1.0,9.0,4.0,18.0,0,0.0,1.0,0.0,4.0,1.0,...,0,0,2,0,0,0,0,0,0,2022-09-09 18:24:03
3,1.0,9.0,4.0,18.0,0,0.0,1.0,0.0,5.0,2.0,...,0,0,1,0,0,0,0,0,0,2022-09-09 18:02:19
4,1.0,9.0,5.0,17.0,0,0.0,1.0,0.0,1.0,1.0,...,0,0,0,0,0,0,1,1,0,2022-09-10 17:15:06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5878,6.0,3.0,1.0,12.0,2,0.0,0.0,0.0,7.0,1258.0,...,0,0,0,1,0,1,1,0,0,2023-03-28 12:54:10
5879,6.0,3.0,1.0,14.0,2,0.0,0.0,0.0,5.0,4620.0,...,0,0,0,0,0,0,1,1,0,2023-03-28 14:16:00
5880,6.0,3.0,1.0,18.0,2,0.0,0.0,0.0,0.0,4621.0,...,0,0,1,0,0,1,1,0,0,2023-03-28 18:38:29
5881,6.0,3.0,4.0,16.0,2,1.0,1.0,1.0,3.0,4622.0,...,0,0,0,0,0,0,1,1,0,2023-03-31 16:19:44


In [15]:
dynamic_context

,avg_irradiation,steering_speed,temperature_out,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,latitude,longitude,altitude,window_id,session,datetime,wind_id
0,0.0,14.890339,20.0,0.0,0.0,26.0,20.442914,48.86775,8.810342,444.0,0,7.0,2022-09-07 20:59:13,0
1,0.0,0.0,20.5,3200.0,768.0,26.5,0.0,48.771829,9.208283,370.0,0,16.0,2022-09-09 18:41:58,1
2,0.0,0.0,20.5,3200.0,774.0,26.5,0.0,48.771828,9.208283,370.0,0,16.0,2022-09-09 18:41:59,1
3,0.0,0.0,20.5,3200.0,774.0,26.5,0.0,48.771827,9.208283,370.0,0,16.0,2022-09-09 18:42:00,1
4,0.0,0.0,20.5,3200.0,780.0,26.5,0.0,48.771826,9.208284,370.0,0,16.0,2022-09-09 18:42:01,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588295,37.789256,14.343311,10.0,9600.0,2427.118215,27.5,72.651963,48.572366,9.590398,735.607726,4623,5399.0,2023-03-31 16:07:06,5882
588296,30.0,6.288697,10.0,8426.938647,2561.191841,27.5,79.379413,48.572408,9.590138,733.607536,4623,5399.0,2023-03-31 16:07:05,5882
588297,40.0,33.722877,10.0,8400.0,2532.656724,27.5,82.957043,48.572456,9.589565,729.607809,4623,5399.0,2023-03-31 16:07:03,5882
588298,30.0,0.0,9.5,10431.1668,3788.805012,27.5,60.292066,48.571514,9.591373,745.607303,4623,5399.0,2023-03-31 16:07:14,5882


In [16]:
static_context.columns

Index(['car_id', 'month', 'weekday', 'hour', 'season', 'seatbelt_codriver',
       'seatbelt_rear_l', 'seatbelt_rear_r', 'street_category', 'window_id',
       'session', 'datetime_x', 'wind_id', 'ess_status', 'current_drive_mode',
       'current_clima_mode', 'current_media_source', 'nav_guidance_status',
       'proximity_to_parking_spot', 'phone_status', 'bluetooth_connected',
       'phone_os', 'new_bluetooth_device_to_pair', 'datetime_y'],
      dtype='object')

In [17]:
dynamic_context.columns

Index(['avg_irradiation', 'steering_speed', 'temperature_out',
       'light_sensor_rear', 'light_sensor_front', 'temperature_in',
       'KBI_speed', 'latitude', 'longitude', 'altitude', 'window_id',
       'session', 'datetime', 'wind_id'],
      dtype='object')

In [18]:
dense_static_context.columns

Index(['wind_id', 'distance_driven_benchmark', 'soc', 'time_second',
       'window_id', 'session', 'datetime'],
      dtype='object')

In [19]:
dynamic_context = dynamic_context.sort_values(by=['wind_id', 'datetime'])
static_context = static_context.sort_values(by=['wind_id'])
dense_static_context = dense_static_context.sort_values(by=['wind_id', 'datetime'])
if feat_engg:
    static_context = static_context.drop(columns=['datetime_y', 'datetime_x'])
else:
    static_context = static_context.drop(columns=['datetime'])

# rearrage order of columns
columns = list(static_context.columns)
columns.remove('car_id')
columns.append('car_id')
static_context = static_context[columns]

dynamic_context = dynamic_context.drop(columns='wind_id')
static_context = static_context.drop(columns=['wind_id'])
dense_static_context = dense_static_context.drop(columns=['wind_id'])

train_dynamic_context = dynamic_context[dynamic_context['session'].isin(train_sessions)].reset_index(drop=True)
test_dynamic_context = dynamic_context[dynamic_context['session'].isin(test_sessions)].reset_index(drop=True)
train_static_context = static_context[static_context['session'].isin(train_sessions)].reset_index(drop=True)
test_static_context = static_context[static_context['session'].isin(test_sessions)].reset_index(drop=True)
train_dense_static_context = dense_static_context[dense_static_context['session'].isin(train_sessions)].reset_index(drop=True)
test_dense_static_context = dense_static_context[dense_static_context['session'].isin(test_sessions)].reset_index(drop=True)

print('number of session', len(train_dynamic_context.window_id.unique().tolist()), len(test_dynamic_context.window_id.unique().tolist()),
       len(train_static_context.window_id.unique().tolist()), len(test_static_context.window_id.unique().tolist()),
       len(train_dense_static_context.window_id.unique().tolist()), len(test_dense_static_context.window_id.unique().tolist()))

train_dynamic_context.to_csv(f'{base_path}{augmentation_folder}dynamic_context/unnormal/train.csv', index=False)
test_dynamic_context.to_csv(f'{base_path}{augmentation_folder}dynamic_context/unnormal/test.csv', index=False)

train_dense_static_context.to_csv(f'{base_path}{augmentation_folder}dense_static_context/unnormal/train.csv', index=False)
test_dense_static_context.to_csv(f'{base_path}{augmentation_folder}dense_static_context/unnormal/test.csv', index=False)

number of session 4624 1259 4624 1259 4624 1259


In [20]:
dynamic_context

,avg_irradiation,steering_speed,temperature_out,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,latitude,longitude,altitude,window_id,session,datetime
0,0.0,14.890339,20.0,0.0,0.0,26.0,20.442914,48.86775,8.810342,444.0,0,7.0,2022-09-07 20:59:13
71,0.0,12.610906,20.0,0.0,0.0,26.0,18.53704,48.867717,8.810246,444.0,0,7.0,2022-09-07 20:59:14
177,0.0,12.610906,20.0,0.0,0.0,26.0,18.53704,48.867717,8.810246,444.0,0,7.0,2022-09-07 20:59:14
128,0.0,24.653542,20.0,0.0,0.0,26.0,16.32,48.867686,8.810158,444.0,0,7.0,2022-09-07 20:59:16
129,0.0,99.878832,20.0,0.0,0.0,26.160953,13.76,48.867667,8.810107,444.0,0,7.0,2022-09-07 20:59:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
588227,30.0,0.0,9.0,5600.0,876.0,27.5,39.04,48.567284,9.605088,718.0,4623,5399.0,2023-03-31 16:08:14
588226,30.0,0.0,9.0,5629.085099,1026.544147,27.5,40.804305,48.567269,9.605212,718.0,4623,5399.0,2023-03-31 16:08:15
588223,22.610694,102.026156,9.0,6370.064358,1773.267177,27.5,42.984007,48.567157,9.605245,716.391266,4623,5399.0,2023-03-31 16:08:16
588271,12.616554,40.0,9.0,6800.0,2124.0,27.5,49.132555,48.567049,9.605246,716.0,4623,5399.0,2023-03-31 16:08:17


In [21]:
# for column in dynamic_context.columns:
#     plt.figure(figsize=(8, 6))
#     sns.histplot(dynamic_context[column], bins=20, color='skyblue', edgecolor='black', label='Dynamic', stat='percent')
#     plt.title(f'Histogram of {column}')
#     plt.xlabel(column)
#     plt.ylabel('Percentage')
#     plt.legend()
#     plt.grid(True)
#     plt.show()

# #distance_driven_benchmark is unevenly distributed hence use from sklearn.preprocessing import RobustScaler. 
# #It scales features using statistics that are robust to outliers. This will scale the column while taking into account the presence of outliers. 

In [22]:
# for column in dense_static_context.columns:
#     plt.figure(figsize=(8, 6))
#     sns.histplot(dense_static_context[column], bins=20, color='skyblue', edgecolor='black', label='dense static', stat='percent')
#     plt.title(f'Histogram of {column}')
#     plt.xlabel(column)
#     plt.ylabel('Percentage')
#     plt.legend()
#     plt.grid(True)
#     plt.show()


In [23]:
# #Normalisation
# dynamic_context_to_normalize = [col for col in train_dynamic_context.columns if col not in ['window_id', 'wind_id', 'session_ids', 'datetime', 'session_id', 'session']]
# scaler_dynamic_context = RobustScaler(unit_variance=True)
# scaler_dynamic_context.fit(train_dynamic_context[dynamic_context_to_normalize])
# train_dc = scaler_dynamic_context.transform(train_dynamic_context[dynamic_context_to_normalize])
# test_dc  = scaler_dynamic_context.transform(test_dynamic_context[dynamic_context_to_normalize])

# scaler_dense_static_context = RobustScaler(unit_variance=True)
# dense_static_context_to_normalize = [col for col in train_dense_static_context.columns if col not in ['window_id', 'wind_id', 'session_ids', 'datetime', 'session_id', 'session']]
# scaler_dense_static_context.fit(train_dense_static_context[dense_static_context_to_normalize])
# train_sdc = scaler_dense_static_context.transform(train_dense_static_context[dense_static_context_to_normalize])
# test_sdc = scaler_dense_static_context.transform(test_dense_static_context[dense_static_context_to_normalize])

# train_dc = pd.DataFrame(train_dc, columns=['avg_irradiation', 'steering_speed', 'temperature_out',
#        'light_sensor_rear', 'light_sensor_front', 'temperature_in',
#        'KBI_speed', 'latitude', 'longitude', 'altitude'])
# test_dc = pd.DataFrame(test_dc, columns=['avg_irradiation', 'steering_speed', 'temperature_out',
#        'light_sensor_rear', 'light_sensor_front', 'temperature_in',
#        'KBI_speed', 'latitude', 'longitude', 'altitude'])
# train_sdc = pd.DataFrame(train_sdc, columns=['distance_driven_benchmark', 'soc', 'time_second'])
# test_sdc = pd.DataFrame(test_sdc, columns=['distance_driven_benchmark', 'soc', 'time_second'])

In [24]:
# for column in test_dc.columns:
#     plt.figure(figsize=(8, 6))
#     sns.histplot(test_dc[column], bins=20, color='skyblue', edgecolor='black', label='Test Data', stat='percent')
#     sns.histplot(train_dc[column], bins=20, color='salmon', edgecolor='black', label='Train Data', stat='percent')
#     plt.title(f'Histogram of {column}')
#     plt.xlabel(column)
#     plt.ylabel('Percentage')
#     plt.legend()
#     plt.grid(True)
#     plt.show()


In [25]:
# for column in test_sdc.columns:
#     plt.figure(figsize=(8, 6))
#     sns.histplot(test_sdc[column], bins=20, color='skyblue', edgecolor='black', label='Test Data', stat='percent')
#     sns.histplot(train_sdc[column], bins=20, color='salmon', edgecolor='black', label='Train Data', stat='percent')
#     plt.title(f'Histogram of {column}')
#     plt.xlabel(column)
#     plt.ylabel('Percentage')
#     plt.legend()
#     plt.grid(True)
#     plt.show()


In [26]:
# #Normalisation
# dynamic_context_to_normalize = [col for col in train_dynamic_context.columns if col not in ['window_id', 'wind_id', 'session_ids', 'datetime', 'session_id', 'session']]
# scaler_dynamic_context = MinMaxScaler()
# scaler_dynamic_context.fit(train_dynamic_context[dynamic_context_to_normalize])
# train_dc = scaler_dynamic_context.transform(train_dynamic_context[dynamic_context_to_normalize])
# test_dc  = scaler_dynamic_context.transform(test_dynamic_context[dynamic_context_to_normalize])

# scaler_dense_static_context = MinMaxScaler()
# dense_static_context_to_normalize = [col for col in train_dense_static_context.columns if col not in ['window_id', 'wind_id', 'session_ids', 'datetime', 'session_id', 'session']]
# scaler_dense_static_context.fit(train_dense_static_context[dense_static_context_to_normalize])
# train_sdc = scaler_dense_static_context.transform(train_dense_static_context[dense_static_context_to_normalize])
# test_sdc = scaler_dense_static_context.transform(test_dense_static_context[dense_static_context_to_normalize])

# train_dc = pd.DataFrame(train_dc, columns=['avg_irradiation', 'steering_speed', 'temperature_out',
#        'light_sensor_rear', 'light_sensor_front', 'temperature_in',
#        'KBI_speed', 'latitude', 'longitude', 'altitude'])
# test_dc = pd.DataFrame(test_dc, columns=['avg_irradiation', 'steering_speed', 'temperature_out',
#        'light_sensor_rear', 'light_sensor_front', 'temperature_in',
#        'KBI_speed', 'latitude', 'longitude', 'altitude'])
# train_sdc = pd.DataFrame(train_sdc, columns=['distance_driven_benchmark', 'soc', 'time_second'])
# test_sdc = pd.DataFrame(test_sdc, columns=['distance_driven_benchmark', 'soc', 'time_second'])

In [27]:
# for column in test_dc.columns:
#     plt.figure(figsize=(8, 6))
#     sns.histplot(test_dc[column], bins=20, color='skyblue', edgecolor='black', label='Test Data', stat='percent')
#     sns.histplot(train_dc[column], bins=20, color='salmon', edgecolor='black', label='Train Data', stat='percent')
#     plt.title(f'Histogram of {column}')
#     plt.xlabel(column)
#     plt.ylabel('Percentage')
#     plt.legend()
#     plt.grid(True)
#     plt.show()


In [28]:
# for column in test_sdc.columns:
#     plt.figure(figsize=(8, 6))
#     sns.histplot(test_sdc[column], bins=20, color='skyblue', edgecolor='black', label='Test Data', stat='percent')
#     sns.histplot(train_sdc[column], bins=20, color='salmon', edgecolor='black', label='Train Data', stat='percent')
#     plt.title(f'Histogram of {column}')
#     plt.xlabel(column)
#     plt.ylabel('Percentage')
#     plt.legend()
#     plt.grid(True)
#     plt.show()

In [29]:
# #Normalisation
# dynamic_context_to_normalize = [col for col in train_dynamic_context.columns if col not in ['window_id', 'wind_id', 'session_ids', 'datetime', 'session_id', 'session']]
# scaler_dynamic_context = StandardScaler()
# scaler_dynamic_context.fit(train_dynamic_context[dynamic_context_to_normalize])
# train_dc = scaler_dynamic_context.transform(train_dynamic_context[dynamic_context_to_normalize])
# test_dc  = scaler_dynamic_context.transform(test_dynamic_context[dynamic_context_to_normalize])

# scaler_dense_static_context = StandardScaler()
# dense_static_context_to_normalize = [col for col in train_dense_static_context.columns if col not in ['window_id', 'wind_id', 'session_ids', 'datetime', 'session_id', 'session']]
# scaler_dense_static_context.fit(train_dense_static_context[dense_static_context_to_normalize])
# train_sdc = scaler_dense_static_context.transform(train_dense_static_context[dense_static_context_to_normalize])
# test_sdc = scaler_dense_static_context.transform(test_dense_static_context[dense_static_context_to_normalize])

# train_dc = pd.DataFrame(train_dc, columns=['avg_irradiation', 'steering_speed', 'temperature_out',
#        'light_sensor_rear', 'light_sensor_front', 'temperature_in',
#        'KBI_speed', 'latitude', 'longitude', 'altitude'])
# test_dc = pd.DataFrame(test_dc, columns=['avg_irradiation', 'steering_speed', 'temperature_out',
#        'light_sensor_rear', 'light_sensor_front', 'temperature_in',
#        'KBI_speed', 'latitude', 'longitude', 'altitude'])
# train_sdc = pd.DataFrame(train_sdc, columns=['distance_driven_benchmark', 'soc', 'time_second'])
# test_sdc = pd.DataFrame(test_sdc, columns=['distance_driven_benchmark', 'soc', 'time_second'])

In [30]:
# for column in test_dc.columns:
#     plt.figure(figsize=(8, 6))
#     sns.histplot(test_dc[column], bins=20, color='skyblue', edgecolor='black', label='Test Data', stat='percent')
#     sns.histplot(train_dc[column], bins=20, color='salmon', edgecolor='black', label='Train Data', stat='percent')
#     plt.title(f'Histogram of {column}')
#     plt.xlabel(column)
#     plt.ylabel('Percentage')
#     plt.legend()
#     plt.grid(True)
#     plt.show()

In [31]:
# for column in test_sdc.columns:
#     plt.figure(figsize=(8, 6))
#     sns.histplot(test_sdc[column], bins=20, color='skyblue', edgecolor='black', label='Test Data', stat='percent')
#     sns.histplot(train_sdc[column], bins=20, color='salmon', edgecolor='black', label='Train Data', stat='percent')
#     plt.title(f'Histogram of {column}')
#     plt.xlabel(column)
#     plt.ylabel('Percentage')
#     plt.legend()
#     plt.grid(True)
#     plt.show()

In [32]:
#Normalisation
dynamic_context_to_normalize = [col for col in train_dynamic_context.columns if col not in ['window_id', 'wind_id', 'session_ids', 'datetime', 'session_id', 'session']]
scaler_dynamic_context = RobustScaler()
scaler_dynamic_context.fit(train_dynamic_context[dynamic_context_to_normalize])
train_dynamic_context[dynamic_context_to_normalize] = scaler_dynamic_context.transform(train_dynamic_context[dynamic_context_to_normalize])
test_dynamic_context[dynamic_context_to_normalize] = scaler_dynamic_context.transform(test_dynamic_context[dynamic_context_to_normalize])

#Robust scaler is used to account for the outlier in the data which make it non uniformly distributed
scaler_dense_static_context = RobustScaler()
scaler_dense_static_context.fit(train_dense_static_context[dense_static_context_var])
train_dense_static_context[dense_static_context_var] = scaler_dense_static_context.transform(train_dense_static_context[dense_static_context_var])
test_dense_static_context[dense_static_context_var] = scaler_dense_static_context.transform(test_dense_static_context[dense_static_context_var])

train_dynamic_context.to_csv(train_dynamic_context_path, index=False)
test_dynamic_context.to_csv(test_dynamic_context_path, index=False)

train_static_context.to_csv(train_static_context_path, index=False)
test_static_context.to_csv(test_static_context_path, index=False)

train_dense_static_context.to_csv(train_dense_static_context_path, index=False)
test_dense_static_context.to_csv(test_dense_static_context_path, index=False)

In [33]:
# for column in train_dynamic_context.columns:
#     print(column)
#     plt.figure(figsize=(8, 6))
#     sns.histplot(train_dynamic_context[column], bins=20, color='skyblue', edgecolor='black')
#     plt.title(f'Histogram of {column}')
#     plt.xlabel(column)
#     plt.ylabel('Frequency')
#     plt.grid(True) 
#     plt.show()

In [34]:
test_dense_static_context

,distance_driven_benchmark,soc,time_second,window_id,session,datetime
0,0.333333,-0.294321,0.111120,0,16.0,2022-09-09 18:43:10
1,0.277778,-0.254155,-0.081701,1,16.0,2022-09-09 18:24:03
2,-0.011111,-0.104571,-0.300916,2,16.0,2022-09-09 18:02:19
3,-0.144444,-0.471607,-0.281247,3,42.0,2022-09-15 09:43:45
4,-0.144444,-0.266620,-0.223754,4,76.0,2022-09-25 12:00:20
...,...,...,...,...,...,...
1254,0.555556,-0.045014,0.218374,1254,5346.0,2023-03-16 09:25:17
1255,0.533333,-0.028393,0.210137,1255,5346.0,2023-03-16 09:24:28
1256,-0.022222,0.423130,-0.021014,1256,5346.0,2023-03-16 09:01:33
1257,-0.122222,0.503463,-0.066235,1257,5346.0,2023-03-16 08:57:04


In [35]:
test_static_context

,month,weekday,hour,season,seatbelt_codriver,seatbelt_rear_l,seatbelt_rear_r,street_category,window_id,session,...,current_drive_mode,current_clima_mode,current_media_source,nav_guidance_status,proximity_to_parking_spot,phone_status,bluetooth_connected,phone_os,new_bluetooth_device_to_pair,car_id
0,9.0,4.0,18.0,0,0.0,1.0,0.0,7.0,0.0,16.0,...,0,0,1,0,0,0,0,0,0,1.0
1,9.0,4.0,18.0,0,0.0,1.0,0.0,4.0,1.0,16.0,...,0,0,2,0,0,0,0,0,0,1.0
2,9.0,4.0,18.0,0,0.0,1.0,0.0,5.0,2.0,16.0,...,0,0,1,0,0,0,0,0,0,1.0
3,9.0,3.0,9.0,0,0.0,0.0,0.0,7.0,3.0,42.0,...,0,0,0,0,0,0,1,1,0,1.0
4,9.0,6.0,11.0,0,1.0,1.0,1.0,2.0,4.0,76.0,...,1,0,0,0,0,0,1,1,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,3.0,3.0,9.0,2,0.0,0.0,0.0,5.0,1254.0,5346.0,...,0,0,0,1,0,0,1,1,0,6.0
1255,3.0,3.0,9.0,2,0.0,0.0,0.0,5.0,1255.0,5346.0,...,0,0,0,1,0,0,1,1,0,6.0
1256,3.0,3.0,9.0,2,0.0,0.0,0.0,7.0,1256.0,5346.0,...,1,0,0,1,0,0,1,1,0,6.0
1257,3.0,3.0,8.0,2,0.0,0.0,0.0,7.0,1257.0,5346.0,...,0,0,0,1,0,0,1,1,0,6.0


In [36]:
test_dynamic_context

,avg_irradiation,steering_speed,temperature_out,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,latitude,longitude,altitude,window_id,session,datetime
0,0.000000,0.000000,0.913043,0.019231,-0.065541,0.333333,-0.262712,-0.018452,0.105133,-0.016807,0,16.0,2022-09-09 18:41:30
1,0.000000,0.000000,0.913043,0.019231,-0.066976,0.333333,-0.262712,-0.018450,0.105134,-0.016807,0,16.0,2022-09-09 18:41:31
2,0.000000,0.000000,0.913043,0.019231,-0.065165,0.333333,-0.262712,-0.018448,0.105134,-0.022548,0,16.0,2022-09-09 18:41:32
3,0.000000,0.000000,0.913043,0.019231,-0.066975,0.333333,-0.262712,-0.018447,0.105136,-0.019468,0,16.0,2022-09-09 18:41:33
4,0.000000,0.000000,0.913043,0.019231,-0.066164,0.333333,-0.262712,-0.018446,0.105137,-0.016807,0,16.0,2022-09-09 18:41:34
...,...,...,...,...,...,...,...,...,...,...,...,...,...
125895,9.655707,0.000000,-0.130435,2.750000,1.220701,0.333333,1.588983,0.053863,-0.066578,-0.201681,1258,5387.0,2023-03-28 12:54:06
125896,9.842094,0.000000,-0.130435,2.750000,1.220701,0.333333,1.588983,0.054273,-0.066233,-0.203787,1258,5387.0,2023-03-28 12:54:07
125897,9.496898,0.937205,-0.130435,2.788462,1.220701,0.333333,1.593220,0.054684,-0.065888,-0.210084,1258,5387.0,2023-03-28 12:54:08
125898,9.339947,0.000000,-0.130435,2.750000,1.220701,0.333333,1.593220,0.055094,-0.065539,-0.212192,1258,5387.0,2023-03-28 12:54:09


In [37]:
train_static_context

,month,weekday,hour,season,seatbelt_codriver,seatbelt_rear_l,seatbelt_rear_r,street_category,window_id,session,...,current_drive_mode,current_clima_mode,current_media_source,nav_guidance_status,proximity_to_parking_spot,phone_status,bluetooth_connected,phone_os,new_bluetooth_device_to_pair,car_id
0,9.0,2.0,21.0,0,1.0,1.0,0.0,1.0,0.0,7.0,...,0,0,0,0,0,0,0,0,0,1.0
1,9.0,5.0,17.0,0,0.0,1.0,0.0,1.0,1.0,20.0,...,0,0,0,0,0,0,1,1,0,1.0
2,9.0,6.0,10.0,0,1.0,1.0,1.0,1.0,2.0,25.0,...,0,0,3,0,0,0,1,1,0,1.0
3,9.0,6.0,12.0,0,1.0,1.0,1.0,7.0,3.0,27.0,...,0,0,0,0,0,0,0,0,0,1.0
4,9.0,0.0,17.0,0,0.0,0.0,0.0,0.0,4.0,33.0,...,0,0,0,0,0,0,1,1,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4619,3.0,1.0,10.0,2,0.0,0.0,0.0,7.0,4619.0,5386.0,...,0,0,0,0,0,0,1,1,0,6.0
4620,3.0,1.0,14.0,2,0.0,0.0,0.0,5.0,4620.0,5388.0,...,0,0,0,0,0,0,1,1,0,6.0
4621,3.0,1.0,18.0,2,0.0,0.0,0.0,0.0,4621.0,5389.0,...,0,0,1,0,0,1,1,0,0,6.0
4622,3.0,4.0,16.0,2,1.0,1.0,1.0,3.0,4622.0,5399.0,...,0,0,0,0,0,0,1,1,0,6.0


In [38]:
train_dense_static_context

,distance_driven_benchmark,soc,time_second,window_id,session,datetime
0,-0.188889,-0.518698,0.084391,0,7.0,2022-09-07 21:00:51
1,-0.211111,-0.294321,-0.383962,1,20.0,2022-09-10 17:15:06
2,-0.211111,-0.378809,-0.423636,2,25.0,2022-09-11 10:28:01
3,-0.211111,-0.385734,-0.432378,3,27.0,2022-09-11 12:26:02
4,-0.200000,0.676593,-0.412709,4,33.0,2022-09-12 17:09:43
...,...,...,...,...,...,...
4619,-0.211111,0.220914,-0.251492,4619,5386.0,2023-03-28 10:10:12
4620,0.366667,-1.628116,-0.132807,4620,5388.0,2023-03-28 14:16:00
4621,1.177778,-0.177978,0.601832,4621,5389.0,2023-03-28 18:38:29
4622,0.244444,-0.327562,-0.041187,4622,5399.0,2023-03-31 16:19:44


In [39]:
train_dynamic_context

,avg_irradiation,steering_speed,temperature_out,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,latitude,longitude,altitude,window_id,session,datetime
0,0.000000,4.806463,0.869565,-0.250000,-0.245693,0.266667,0.007984,0.116007,-0.424283,0.285714,0,7.0,2022-09-07 20:59:13
1,0.000000,4.070683,0.869565,-0.250000,-0.245693,0.266667,-0.017253,0.115962,-0.424412,0.285714,0,7.0,2022-09-07 20:59:14
2,0.000000,4.070683,0.869565,-0.250000,-0.245693,0.266667,-0.017253,0.115962,-0.424412,0.285714,0,7.0,2022-09-07 20:59:14
3,0.000000,7.957934,0.869565,-0.250000,-0.245693,0.266667,-0.046610,0.115919,-0.424529,0.285714,0,7.0,2022-09-07 20:59:16
4,0.000000,32.239959,0.869565,-0.250000,-0.245693,0.288127,-0.080508,0.115891,-0.424596,0.285714,0,7.0,2022-09-07 20:59:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
462395,1.000000,0.000000,-0.086957,0.288462,-0.036003,0.466667,0.254237,-0.305135,0.633101,1.436975,4623,5399.0,2023-03-31 16:08:14
462396,1.000000,0.000000,-0.086957,0.291258,0.000033,0.466667,0.277599,-0.305156,0.633265,1.436975,4623,5399.0,2023-03-31 16:08:15
462397,0.753690,32.933095,-0.086957,0.362506,0.178778,0.466667,0.306462,-0.305314,0.633310,1.430215,4623,5399.0,2023-03-31 16:08:16
462398,0.420552,12.911628,-0.086957,0.403846,0.262733,0.466667,0.387878,-0.305465,0.633310,1.428571,4623,5399.0,2023-03-31 16:08:17


In [40]:
def session_window_mapping(df):
    session_window_dict = {}

    for index, row in df.iterrows():
        session = int(row['session'])
        window_id = int(row['window_id'])
        
        # If the session is not already in the dictionary, initialize an empty set
        if session not in session_window_dict:
            session_window_dict[session] = set()
        
        # Add the window_id to the set corresponding to the session
        session_window_dict[session].add(window_id)

    # Convert sets to lists in the resulting dictionary
    session_window_dict = {session: list(window_ids) for session, window_ids in session_window_dict.items()}
    return session_window_dict

train_session_win_id_mapping_dc = session_window_mapping(train_dynamic_context)
test_session_win_id_mapping_dc = session_window_mapping(test_dynamic_context)
train_session_win_id_mapping_sc = session_window_mapping(train_static_context)
test_session_win_id_mapping_sc = session_window_mapping(test_static_context)
train_session_win_id_mapping_dsc = session_window_mapping(train_dense_static_context)
test_session_win_id_mapping_dsc = session_window_mapping(test_dense_static_context)

with open(os.path.join(parameter_path, 'session_win_id_mapping.pkl'), 'rb') as pickle_file:
    train_session_win_id_mapping = pickle.load(pickle_file)
    test_session_win_id_mapping = pickle.load(pickle_file)

In [41]:
print(len(train_session_win_id_mapping_dc), len(train_session_win_id_mapping_sc),  len(train_session_win_id_mapping_dsc), len(train_session_win_id_mapping))

1308 1308 1308 1308


In [42]:
if train_session_win_id_mapping_dc == train_session_win_id_mapping_sc == train_session_win_id_mapping == train_session_win_id_mapping_dsc:
    print("All training data mapping are exactly identical.")
if test_session_win_id_mapping_dc == test_session_win_id_mapping_sc == test_session_win_id_mapping == test_session_win_id_mapping_dsc:
    print("All testing data mapping are exactly identical.")

All training data mapping are exactly identical.
All testing data mapping are exactly identical.


In [43]:
print(len(train_dynamic_context.session.unique().tolist()), len(test_dynamic_context.session.unique().tolist()))
print(len(train_dynamic_context.window_id.unique().tolist()), len(test_dynamic_context.window_id.unique().tolist()))
print(len(train_static_context.session.unique().tolist()), len(test_static_context.session.unique().tolist()))
print(len(train_static_context.window_id.unique().tolist()), len(test_static_context.window_id.unique().tolist()))
print(len(train_dense_static_context.session.unique().tolist()), len(test_dense_static_context.session.unique().tolist()))
print(len(train_dense_static_context.window_id.unique().tolist()), len(test_dense_static_context.window_id.unique().tolist()))

1308 326
4624 1259
1308 326
4624 1259
1308 326
4624 1259
